# Candlesticks Combined patterns analysis

In [3]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")
df.tail()

,Local time,open,high,low,close,volume
6627,26.06.2021 00:00:00.000 GMT+0300,1.19392,1.19392,1.19392,1.19392,0.00000
6628,27.06.2021 00:00:00.000 GMT+0300,1.19392,1.19392,1.19392,1.19392,0.00000
6629,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,85154.26000
6630,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,98898.57000
6631,30.06.2021 00:00:00.000 GMT+0300,1.18973,1.19092,1.18452,1.18589,4301.30191


In [6]:
#Check if NA values are in data
df.isna().sum()
df= df[df['volume'] != 0]
df.reset_index(drop=True, inplace=True)

import pandas_ta as ta
#df.ta.indicators()
#help(ta.atr)
#df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.tail()

,Local time,open,high,low,close,volume,RSI
4729,24.06.2021 00:00:00.000 GMT+0300,1.19267,1.19565,1.19178,1.19322,85152.21000,37.211236
4730,25.06.2021 00:00:00.000 GMT+0300,1.19322,1.19754,1.19264,1.19392,77837.64500,38.068450
4731,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,85154.26000,37.046582
4732,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,98898.57000,34.866329
4733,30.06.2021 00:00:00.000 GMT+0300,1.18973,1.19092,1.18452,1.18589,4301.30191,32.153632


In [7]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
    
    
def isEngulfingStrong(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<low[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>high[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
    
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0

def direction(l):
    if (open[l]>close[l]):
        return 1
    elif (open[l]<close[l]):
        return 2
    else:
        return 0
    

# Identifying the candle signal

In [28]:
def Revsignal1():
    signal = [0] * length
    for row in range(1, length):
        if ((isEngulfing(row)==1 and isStar(row)==1) ):#and df.RSI[row]<30
            signal[row] = 1
        elif((isEngulfing(row)==2 and isStar(row)==2) ):#and df.RSI[row]>70
            signal[row] = 2
        else:
            signal[row] = 0
    return signal

df['signal'] = Revsignal1()
df.tail()

,Local time,open,high,low,close,volume,RSI,signal,Trend,result
4729,24.06.2021 00:00:00.000 GMT+0300,1.19267,1.19565,1.19178,1.19322,85152.21000,37.211236,0,NaN,0
4730,25.06.2021 00:00:00.000 GMT+0300,1.19322,1.19754,1.19264,1.19392,77837.64500,38.068450,0,NaN,0
4731,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,85154.26000,37.046582,0,NaN,0
4732,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,98898.57000,34.866329,0,NaN,0
4733,30.06.2021 00:00:00.000 GMT+0300,1.18973,1.19092,1.18452,1.18589,4301.30191,32.153632,0,NaN,0


In [30]:
#df.reset_index(drop=True, inplace=True)
df[df['signal']==2].count()

Local time    4
open          4
high          4
low           4
close         4
volume        4
RSI           4
signal        4
Trend         4
result        4
dtype: int64

In [24]:
#Target
def mytarget(df1, barsfront):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    trendcat = [None] * length
    
    piplim = 200e-5
    for line in range (0, length-1-barsfront):
        for i in range(1,barsfront+1):
            if( ((high[line+i]-close[line])>piplim) and ((close[line]-low[line+i])>piplim) ):
                trendcat[line] = 3 # no trend
                break
            elif(close[line]-low[line+i])>piplim:
                trendcat[line] = 1 #-1 downtrend
                break
            elif ((high[line+i]-close[line])>piplim):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend  
    return trendcat

df['Trend'] = mytarget(df,10)
#df.head(30)

In [26]:
import numpy as np
conditions = [( (df['Trend'] == 1) | (df['Trend'] == 3) ) & (df['signal'] == 1), ((df['Trend'] == 2) | (df['Trend'] == 3)) & (df['signal'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=1
print(df[df['result']==trendId].result.count()/df[df['signal']==trendId].signal.count())
df[ (df['Trend']!=trendId) & (df['Trend']!=3) & (df['signal']==trendId) ] # false positives

0.8235294117647058


,Local time,open,high,low,close,volume,RSI,signal,Trend,result
2969,19.09.2014 00:00:00.000 GMT+0300,1.29240,1.29292,1.28288,1.28301,208346.7396,28.228790,1,2.0,0
2979,03.10.2014 00:00:00.000 GMT+0300,1.26691,1.26754,1.25008,1.25158,188134.2992,21.032260,1,2.0,0
3056,20.01.2015 00:00:00.000 GMT+0200,1.16072,1.16153,1.15407,1.15506,145923.1308,22.011352,1,2.0,0


In [27]:
dfpl = df[2960:3000]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()